In [1]:
from my_tokenizers2 import SmilesTokenizer
import numpy as np
import torch
tokenizer = SmilesTokenizer(vocab_file ='mol2vec_vocabs.txt')


2022-01-14 16:34:32.186735: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [2]:
from transformers import ElectraModel, ElectraConfig,ElectraForPreTraining,ElectraForMaskedLM
emb_dim = 256
output_dir="./fingerprints_smile_output256/full"
electra_model = ElectraForMaskedLM.from_pretrained(output_dir)

In [6]:
import numpy as np
import csv 
trfile = open('../original-dataset/regression/ESOL-MoleculeNet.csv', 'r')
reader = csv.reader(trfile)
next(reader)
smiles,dataY,all_smiles = [], [], []
for raw in reader:
    # print(raw)
    dataY.append(float(raw[-2]))
    smiles = str(raw[-1])
    all_smiles.append(smiles)


    

In [7]:
from collections import Counter, defaultdict
def duplicates(lst):
    cnt= Counter(lst)
    return [key for key in cnt.keys() if cnt[key]> 1]
dups = duplicates(all_smiles)


In [8]:
print(len(dups))

0


In [9]:
newY = []
newSm = []

"""
for i,x in enumerate(all_smiles):
    if x not in dups:
        newY.append(dataY[i])
        newSm.append(x)
"""
for i,x in enumerate(all_smiles):
    newY.append(dataY[i])
    newSm.append(x)

In [10]:
#after removing duplications
print(len(newSm))
print(len(newY))

1128
1128


In [11]:
from tqdm import tqdm
from rdkit import Chem
from mol2vec.features import mol2alt_sentence

#smiles_seqs_emb = {}
#smiles_aug_dict = {}

embX = []
MAX_SMI_LEN = 256

for item in tqdm(newSm):
    
    smiles = item
    aas = Chem.MolFromSmiles(item) 
    sentence = mol2alt_sentence(aas, 1)
    sentence = ' '.join(map(str, sentence)) 

    if len(item) > MAX_SMI_LEN:
        input_ids = tokenizer(sentence[:MAX_SMI_LEN], return_tensors="pt")["input_ids"]
    else:
        input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
    outputs = electra_model(input_ids, labels=input_ids)
    #smiles_seqs_emb[smiles] = outputs[2][0].detach().numpy()[0][1:-1]
    embX.append(outputs[2][0].detach().numpy()[0][1:-1])
#         if item not in smiles_aug_dict.keys():
#             smiles_aug_dict[item] = smiles
#         else:
#             smiles_aug_dict[item] =smiles_aug_dict[item] +';'+ smiles
#embX[0].shape            
        

        
        

[16:35:34] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [16:35:34] Enabling RDKit 2019.09.3 jupyter extensions
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1128/1128 [00:20<00:00, 54.01it/s]


In [13]:
np.save('../encoded-dataset/regression/ESOL_embX_fullBERT', embX)
np.save('../encoded-dataset/regression/ESOL_Y_fullBERT', newY)